In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../data/TSLA.csv')

In [ ]:
df.head(15)

In [ ]:
df.isna().sum()

In [ ]:
def read_data(file_path, features_importance=None):
    data = pd.read_csv(file_path)

    # Lag Features
    data['lag_1'] = data['Close'].shift(1)
    data['lag_5'] = data['Close'].shift(5)
    data['lag_10'] = data['Close'].shift(10)

    # Rolling Mean Features
    data['rolling_mean_5'] = data['Close'].rolling(window=5).mean()
    data['rolling_mean_10'] = data['Close'].rolling(window=10).mean()
    data['rolling_mean_20'] = data['Close'].rolling(window=20).mean()

    # Rolling Std Features
    data['rolling_std_5'] = data['Close'].rolling(window=5).std()
    data['rolling_std_10'] = data['Close'].rolling(window=10).std()
    data['rolling_std_20'] = data['Close'].rolling(window=20).std()

    # Expanding Mean Feature
    data['expanding_mean'] = data['Close'].expanding().mean()

    # Xử lý giá trị thiếu và vô hạn
    
    data = data.replace([np.inf, -np.inf], 0)
    

    data.drop(columns=['High', 'Low', 'Open', 'Volume'], inplace=True)
    data['ParabolicSAR'] = data['ParabolicSAR'].shift(1)
    data['TrueRange'] = data['TrueRange'].shift(1)
    
    if features_importance is not None:
        features_importance.append('Close')
        features_importance.append('Date')
        data = data[features_importance]

    data.fillna(method='bfill', inplace=True)
    
    return data


In [ ]:
def prepare_data(stock_data, sequence_length=10, target_column='Close', train_size=325, test_size=125, step=125):
    """
    Chuẩn bị dữ liệu cho mô hình LSTM và XGBoost, bao gồm chuẩn hóa và dịch chuyển tập huấn luyện, kiểm tra.
    
    Parameters:
    -----------
    stock_data : pandas DataFrame
        Dữ liệu cổ phiếu đã được xử lý
    sequence_length : int
        Độ dài chuỗi thời gian cho LSTM
    target_column : str
        Tên cột dữ liệu mục tiêu cần dự đoán
    train_size : int
        Số lượng mẫu dành cho tập huấn luyện
    test_size : int
        Số lượng mẫu dành cho tập kiểm tra
    step : int
        Số lượng mẫu dịch chuyển giữa các lần huấn luyện
        
    Returns:
    --------
    List chứa các bộ dữ liệu huấn luyện và kiểm tra đã xử lý cho LSTM và XGBoost
    """
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data = stock_data.sort_values('Date').reset_index(drop=True)
    datasets = []
    
    for start_idx in range(0, len(stock_data) - (train_size + test_size), step):
        df_train = stock_data.iloc[start_idx:start_idx + train_size]
        df_test = stock_data.iloc[start_idx + train_size - sequence_length:start_idx + train_size + test_size]
        
        feature_columns = [col for col in stock_data.columns if col not in ['Date', target_column]]
        
        X_train = df_train[feature_columns].values
        X_test = df_test[feature_columns].values
        y_train = df_train[target_column].values
        y_test = df_test[target_column].values
        
        # Chuẩn hóa dữ liệu
        scaler_X = MinMaxScaler(feature_range=(0, 1))
        scaler_y = MinMaxScaler(feature_range=(0, 1))
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))
        
        # Chuẩn bị dữ liệu cho XGBoost
        X_train_xgb = X_train_scaled
        X_test_xgb = X_test_scaled
        y_train_xgb = y_train_scaled.flatten()
        y_test_xgb = y_test_scaled.flatten()
        
        # Chuẩn bị dữ liệu cho LSTM
        X_sequences, y_sequences = [], []
        for i in range(len(X_train_scaled) - sequence_length):
            X_sequences.append(X_train_scaled[i:i + sequence_length])
            y_sequences.append(y_train_scaled[i + sequence_length])
        
        X_sequences = np.array(X_sequences)
        y_sequences = np.array(y_sequences)
        
        X_test_sequences, y_test_sequences = [], []
        for i in range(len(X_test_scaled) - sequence_length):
            X_test_sequences.append(X_test_scaled[i:i + sequence_length])
            y_test_sequences.append(y_test_scaled[i + sequence_length])
        
        X_test_sequences = np.array(X_test_sequences)
        y_test_sequences = np.array(y_test_sequences)
        
        datasets.append({
            'xgboost': {
                'X_train': X_train_xgb, 
                'y_train': y_train_xgb,
                'X_test': X_test_xgb[sequence_length:], 
                'y_test': y_test_xgb[sequence_length:],
                'feature_names': feature_columns,
                'scaler_X': scaler_X,
                'scaler_y': scaler_y
            },
            'lstm': {
                'X_train': X_sequences, 
                'y_train': y_sequences,
                'X_test': X_test_sequences, 
                'y_test': y_test_sequences,
                'scaler_y': scaler_y,
                'scaler_X': scaler_X,
                'sequence_length': sequence_length
            },
            'dates_test': df_test['Date'].values[sequence_length:],
            'actual_test': y_test[sequence_length:]
        })
    
    return datasets


In [ ]:
def build_xgboost_model(prepared_data, max_depth=7, learning_rate=0.1, n_estimators=100, early_stopping_rounds=10):
    """
    Xây dựng và huấn luyện mô hình XGBoost với dữ liệu đã chuẩn hóa
    
    Parameters:
    -----------
    prepared_data : dict
        Dictionary chứa dữ liệu đã chuẩn bị cho XGBoost
    max_depth : int
        Độ sâu tối đa của cây
    learning_rate : float
        Tốc độ học của mô hình
    n_estimators : int
        Số lượng cây ước lượng
    early_stopping_rounds : int
        Số vòng đợi trước khi early stopping
        
    Returns:
    --------
    Mô hình XGBoost đã huấn luyện và kết quả dự đoán
    """
    # Lấy dữ liệu
    X_train = prepared_data['xgboost']['X_train']
    y_train = prepared_data['xgboost']['y_train']
    X_test = prepared_data['xgboost']['X_test']
    feature_names = prepared_data['xgboost']['feature_names']
    scaler_y = prepared_data['xgboost']['scaler_y']
    
    X_train_xgb, X_val_xgb, y_train_xgb, y_val_xgb = train_test_split(
        X_train, y_train, test_size=0.1, random_state=42
    )
    
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        random_state=42,
        n_jobs=-1
    )

    model.fit(
        X_train_xgb, y_train_xgb,
        eval_set=[(X_val_xgb, y_val_xgb)],
        verbose=False
    )
    
    y_pred_scaled = model.predict(X_test)

    y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    feature_importance = model.feature_importances_
    feature_importance_dict = {feature: importance for feature, importance in zip(feature_names, feature_importance)}
    
    return {
        'model': model,
        'predictions': y_pred,
        'predictions_scaled': y_pred_scaled,
        'feature_importance': feature_importance_dict
    }